# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

In [ ]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [ ]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    for j in range(len(data)):
        all_data.append(data[j])
        all_target.append(target[j])
all_data = np.asarray(all_data)
print all_data.shape
print 'total number of data', len(all_data)


In [ ]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
test_data = permutation[n_training_data:]
print training_data

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [ ]:
clf = svm.SVC(gamma=0.001, C=100.)

#from sklearn.neural_network import MLPClassifier
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                    hidden_layer_sizes=(25, 25), random_state=1)

### learning

In [116]:
# YOUR CODE HERE
clf.fit(np.asarray([all_data[i] for i in training_data]), np.asarray([all_target[i] for i in training_data])) 

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25, 25), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

### predicting

In [117]:
print np.asarray([all_data[-1]]).shape
clf.predict(all_data[-1:]), all_target[-1]

(1, 10)


(array([10]), 10)

In [118]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [119]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data)
expected = all_target
evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        20
          1       1.00      1.00      1.00        52
          2       1.00      1.00      1.00        30
          3       1.00      1.00      1.00        19
          4       1.00      1.00      1.00        26
          5       1.00      1.00      1.00        10
          6       1.00      1.00      1.00        11
          7       1.00      1.00      1.00        11
          8       1.00      1.00      1.00        10
          9       1.00      1.00      1.00        23
         10       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00       222


Confusion matrix:
[[20  0  0  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0  0]
 [ 0  0  0  0 26  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0 11  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [120]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict([all_data[i] for i in test_data])
expected = np.asarray([all_target[i] for i in test_data])

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         8
          1       1.00      1.00      1.00        17
          2       1.00      1.00      1.00         4
          3       1.00      1.00      1.00         6
          4       1.00      1.00      1.00        10
          5       1.00      1.00      1.00         3
          7       1.00      1.00      1.00         3
          8       1.00      1.00      1.00         5
          9       1.00      1.00      1.00         8
         10       1.00      1.00      1.00         3

avg / total       1.00      1.00      1.00        67


Confusion matrix:
[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 17  0  0  0  0  0  0  0  0]
 [ 0  0  4  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0  0  0  3  0  0  0]
 [ 0  0  0  0  0  0  0  5  0  0]
 [ 0  0  0  0  0  0  0  0  8  0]
 [ 0  0  0  0  0  0  0

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [121]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [122]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1]

(array([10]), 10)